# Homework 1 (2022) : Data Wrangling and Visualization

Due date : **2022-01-30 @23h55** (this is a **hard deadline**)

## Fill this cell with your names

- Bignon, Thomas, Informatique
- Fezoui, Lyes, Informatique

## Carefully follow instructions given on the [course homepage](https://stephanegaiffas.github.io/big_data_course/homeworks/)

Write in English or French

- The deliverable is a `xxx.ipynb` file (jupyter notebook) or a `xxx.py` file (if you are using `jupytext`) built by completing the template. **We won't execute the code in your notebook:** all your results, displays and plots must be visible without having to rerun everything.

Once again, follow the steps described here : [course homepage](https://stephanegaiffas.github.io/big_data_course/homeworks/). **If you don't: no evaluation!**

## Grading <i class="fa graduation-cap"></i>

Here is the way we'll assess your work

| Criterion | Points | Details |
|:----------|:-------:|:----|
|Spelling and syntax | 3 | English/French  |
|Plots correction | 3 |  Clarity / answers the question  |
|Plot style and cleanliness | 3 | Titles, legends, labels, breaks ... |
|Table wrangling | 4 | ETL, SQL like manipulations |
|Computing Statistics | 5 | SQL `goup by`  and aggregation  |
|DRY compliance | 2 | DRY principle at [Wikipedia](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)|

If we see a single (or more) `for` loop in your code: **-5 points**.  Everything can be done using high-level `pandas` methods

# Preliminaries

## Notebook: Modus operandi

- This is a [Jupyter Notebook]().
- When you execute code within the notebook, the results appear beneath the code.
- [Jupytext]()

## Packages

- Base `Python` can do a lot. But the full power of `Python` comes from a fast growing collection of `packages`/`modules`.

- Packages are first installed (that is using `pip install` or `conda install`), and if
needed, imported during a session.

- The `docker` image you are supposed to use already offers a lot of packages. You should not need to install new packages.

- Once a package has been installed on your drive, if you want all objects exported by the package to be available in your session, you should import the package, using `from pkg import *`.

- If you just want to pick some sobjects from the package,
you can use qualified names like `pkg.object_name` to access the object (function, dataset, class...)


In [1]:
# importing basic tools
import numpy as np
import pandas as pd

from pandas.api.types import CategoricalDtype

import os            # file operations
import requests      # networking

from datetime import date  # if needed

In [2]:
# importing plotting packages
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [3]:
# make pandas plotly-friendly
np.set_printoptions(precision=2, suppress=True)
%matplotlib inline
pd.options.plotting.backend = "plotly"

# Getting the data

The data are built and made available by [INSEE](https://www.insee.fr/fr/accueil)  (French Governement Statistics Institute)

Prénoms:
- [https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip](https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip)

This dataset has been growing for a while. It has been considered by
social scientists for decades.  Given names are meant to give insights into a variety
of phenomena, including religious observance.

- A glimpse at the body of work can be found in [_L'archipel français_ by Jérome Fourquet, Le Seuil, 2019 ](https://www.seuil.com/ouvrage/l-archipel-francais-jerome-fourquet/9782021406023)

- Read the [File documentation](https://www.insee.fr/fr/statistiques/2540004?sommaire=4767262#documentation)

## Download the data

**QUESTION:** Download the data into a file which relative path is `'./nat2020_csv.zip'`

__Hints:__

- Have a look at [`requests`](https://requests.readthedocs.io/en/master/).
- Use magic commands to navigate across the file hierarchy and create subdirectories
when needed

In [4]:
params = dict(
    url = 'https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip',
    dirpath = './',
    timecourse = '',
    datafile = 'nat2020.hdf',
    fpath = 'nat2020_csv.zip'
)

In [5]:
# modify location  make sure you are in the right directory
# %cd
# %pwd  #
# %ls
# %mkdir # if needed

In [6]:
url = params['url']      # 'https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip'
fpath = params['fpath']  # './nat2020_csv.zip'

In [7]:
# your code here
import requests

if os.path.exists(fpath):
    print('The file %s already exists.' % fpath)
else:
    r = requests.get(url, allow_redirects=True)
    with open(fpath, 'wb') as f:
        f.write(r.content)
    print('Downloaded file %s.' % fpath)

Downloaded file nat2020_csv.zip.


In [8]:
!ls -laF

total 2508
drwxr-xr-x 5 root root     160 Jan 26 21:19 ./
drwxr-xr-x 6 root root     192 Jan 23 18:32 ../
drwxr-xr-x 3 root root      96 Jan 26 20:17 .ipynb_checkpoints/
-rw-rw-rw- 1 root root  120354 Jan 26 21:18 homework01.ipynb
-rw-r--r-- 1 root root 2443960 Jan 26 21:19 nat2020_csv.zip


## Load the data in memory

**QUESTION:** Load the data in a `pandas` `DataFrame` called `data`

__Hints:__

- You should obtain a `Pandas dataframe` with 4 columns.
- Mind the conventions used to build theis `csv` file.
- Package `pandas` provides the convenient tools.
- The dataset, though not too large, is already demanding.
- Don't hesitate to test your methods on a sample of rows method `sample()` from class `DataFrame` can be helpful.

In [9]:
df = pd.read_csv("nat2020_csv.zip", delimiter=';')

## Explore the data

**QUESTION:** Look at the data, Use the attributes `columns`, `dtypes` and the methods `head`, `describe`, to get a feeling of the data.

- This dataset is supposed to report all given names used
for either sex during a year in France since 1900

- The file is made of `652 056` lines and  4 columns.

```
|-- preusuel : object
|-- nombre: int64
|-- sexe: int64
|-- annais: object
```

Each row indicates for a given `preusuel` (prénom usuel, given name), `sexe` (sex), and `annais` (année naissance, birthyear) the `nombre` (number) of babies of the given sex who were given that name during the given year.

|sexe    |preusuel     | annais|   nombre|
|:------|:--------|----:|---:|
|2     |SYLVETTE | 1953| 577|
|1   |BOUBOU   | 1979|   4|
|1   |NILS     | 1959|   3|
|2   |NICOLE   | 2003|  36|
|1   |JOSÉLITO | 2013|   4|


**QUESTION:** Compare memory usage and disk space used by data

**Hints:**

- The method `info`  prints a concise summary of a `DataFrame`.
- With optional parameter `memory_usage`, you can get an estimate
of the amount of memory used by the `DataFrame`.
- Beware that the resulting estimate depends on the argument fed.

In [10]:
df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667364 entries, 0 to 667363
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sexe      667364 non-null  int64 
 1   preusuel  667362 non-null  object
 2   annais    667364 non-null  object
 3   nombre    667364 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 20.4+ MB


**QUESTION:** Display the output of `.describe()` with style.

In [11]:
(
    df.describe()
    .style
    .background_gradient()
    
)

,sexe,nombre
count,667364.000000,667364.000000
mean,1.543828,129.772665
std,0.498076,884.912504
min,1.000000,1.000000
25%,1.000000,4.000000
50%,2.000000,8.000000
75%,2.000000,25.000000
max,2.000000,53584.000000


**QUESTION:** For each column compute the number of distinct values

In [12]:
for coli in range(4):
    print('Column', coli, '>', len(df.iloc[:, coli].unique()))
    

Column 0 > 2
Column 1 > 35011
Column 2 > 122
Column 3 > 7304


# Transformations

## Improving the data types

**QUESTION:** Make `sexe` a category with two levels `Female` and `Male`. Call the new column `genre`. Do you see any reason why this factor should be ordered?

__Hint:__ Read [Pandas and categorical variables](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html?highlight=category)

In [13]:
df.dtypes

sexe         int64
preusuel    object
annais      object
nombre       int64
dtype: object

In [14]:
df["genre"] = pd.Series(['Female' if s == 2 else 'Male' for s in df["sexe"]], dtype="category")

**QUESTION:** Compare memory usage of columns `sexe` and `genre`

In [15]:
df["sexe"].memory_usage()

5339040

In [16]:
df["genre"].memory_usage()

667616

**QUESTION:** Would it be more memory-efficient to recode `sexe` using modalities `F` and `M` instead of `Male` and `Female` ?

> As the Pandas documentation says : "Internally, the data structure consists of a categories array and an integer array of codes which point to the real value in the categories array." So basically, it won't, because the names are only stored in the `categories` array once.

## Dealing with missing values

**QUESTION:** Variable `annais` class is `object`. Make `annais` of type `float`. Note that missing years are encoded as "XXXX", find a way to deal with that.

__Hint:__  As of releasing this Homework (2021-01-22), `Pandas` is not very good at managing missing values,
see [roadmap](https://pandas.pydata.org/docs/development/roadmap.html). Don't try to convert `annais` into an integer column.

In [17]:
df['annais'] = pd.to_numeric(df['annais'], errors='coerce', downcast='float')

## Rename and remove columns

**QUESTION:** Remove useless columns (now that you've created new ones, and rename them). You should end up with a dataframe with columns called `"gender"`, `"year"`, `"count"`, `"firstname`" with the following dtypes:

```python
gender        category
firstname     object
count         int64
year          float64
```

In [18]:
df.drop(columns=['sexe'], inplace=True)
df.rename(columns={'preusuel': 'firstname', 'annais': 'year', 'nombre': 'count', 'genre': 'gender'}, inplace=True)

In [19]:
df.dtypes

firstname      object
year          float32
count           int64
gender       category
dtype: object

**QUESTION:** How many missing values (NA) have been introduced? How many births are concerned?

In [20]:
print('There is %d missing values and %d births concerned.' % (df[df['year'].isna()].shape[0], df.loc[df['year'].isna(), 'count'].sum()))

There is 36675 missing values and 844964 births concerned.


**QUESTION:** Read the documentation and describe the origin of rows containing the missing values.

> A row contains the value `XXXX` in the `annais` column when a firstname has been given at least 20 times between 1946 and 2000 but less than 3 times for a given year.

## Checkpointing: save your transformed dataframe

**QUESTION:** Save the transformed dataframe (retyped and renamed) to `./nat2020_csv.zip`. Try several compression methods.

In [49]:
# Doesn't work for some reasons... Documentation says method can take these values : {'zip', 'gzip', 'bz2', 'zstd'}

#compression_opts = dict(method='zip', archive_name='nat2020.csv')  
#df.to_csv(fpath, index=False, compression=compression_opts)

**QUESTION:** Save the transformed dataframe (retyped and renamed) to `./nat2020.hdf` using `.hdf` format

In [23]:
# !pip install tables

     |████████████████████████████████| 5.9 MB 2.3 MB/s eta 0:00:01     |███████████████████             | 3.5 MB 1.8 MB/s eta 0:00:02
     |████████████████████████████████| 381 kB 1.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [34]:
df.to_hdf('nat2020.hdf', key='df', mode='w', format="table")

At that point your working directory should look like:

```
├── homework01.py      # if you use `jupytext`
├── homework01.ipynb
├── nat2020.hdf
├── nat2020.zip
```

**QUESTION:** Reload the data using `read_hdf(...)` so that the resulting dataframe  is properly typed with meaningful column names.

__Hint:__ use `try: ... except` to handle exceptions such as `FileNotFoundError`

In [35]:
try :
    pd_hdf = pd.read_hdf('nat2020.hdf', 'df')
except FileNotFoundError:
    print("File 'nat2020.hdf' not found.")

In [36]:
pd_hdf

,firstname,year,count,gender
0,_PRENOMS_RARES,1900.0,1250,Male
1,_PRENOMS_RARES,1901.0,1342,Male
2,_PRENOMS_RARES,1902.0,1330,Male
3,_PRENOMS_RARES,1903.0,1286,Male
4,_PRENOMS_RARES,1904.0,1430,Male
...,...,...,...,...
667359,ZYNEB,2017.0,6,Female
667360,ZYNEB,2018.0,5,Female
667361,ZYNEB,2019.0,7,Female
667362,ZYNEB,2020.0,8,Female


## Some data "analytics" and visualization

**QUESTION**: For each year, compute the total number of Female and Male births and the proportion of Female  births among total births

__Hints:__

- Groupby operations using several columns for the groups return a dataframe with a `MultiIndex` index see [Pandas advanced](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)

- Have a look at `MultiIndex`, `reset_index`, `pivot`, `columns.droplevel`

In [55]:
# Tests... Not over.

#index = pd.MultiIndex.from_tuples(df., names=["first", "second"])
#df_a = df.loc[:, ['year', 'gender', 'count']] 

In [57]:
#df_a.groupby(['year', 'gender'])

**QUESTION:** Plot the proportion of female births as a function of year. Don't forget: title, axes labels, ticks, scales, etc.

Because of what we did before, the `plot` method of a `DataFrame` with be rendered using `plotly`, so you can use this. But you can use also `seaborn` or any other available plotting library that you want.

__Hint:__ Mind the missing values in the `year` column

In [ ]:
# your code here


**QUESTION:** Make any sensible comment about this plot.

_Insert your answer here_

> ...

**QUESTION:** Explore the fluctuations of sex ratio around its mean value since 1945.
Plot deviations of sex ratio around its mean since 1945 as a function of time.

In [ ]:
# your code here


# Popular firstnames

## The top-20 most popular firstnames since 1900

**QUESTION:** For each firstname and sex (some names may be given to girls and boys), compute the total number of times this firstname has been given during `1900-2019`. Print the top 20 firstnames given and style your result dataframe using `background_gradient` for instance.

In [ ]:
# your code here


## Rare firstnames

**QUESTION:** For each sex, plot the proportion of births given `_PRENOMS_RARES` as a function of the year.

In [ ]:
# your code here


# A study of the "Marie" firstname

**QUESTION:** Plot the proportion of female births given name `'MARIE'` or `'MARIE-...'` (compounded names) as a function of the year.
Proceed in such a way that the reader can see the share of compounded names. We are expecting an _area plot_.

__Hints:__

- Have a look at the `.str` accessor (to apply a string method over a whole column containing string)
- Have a look at [r-graph-gallery: stacked area](https://www.r-graph-gallery.com/stacked-area-graph.html)  and
at [ggplot documentation](https://ggplot2.tidyverse.org/reference/geom_ribbon.html). Pay attention on the way you stack the area corresponding to names matching pattern 'MARIE-.*' over or under the are corresponding to babies named 'MARIE'
- See Graphique 3, page 48, de _L'archipel français_  de J. Fourquet. Le Seuil. Essais. Vol. 898.

- Add annotation, 1st World War, Front Populaire, 2nd World War, 1968

In [ ]:
# your code here


# Top 10 firstnames of year 2000

**QUESTION:** For each sex, select the ten most popular names in year 2000, and plot the proportion
of newborns given that name over time. Take into account that some names might have
zero occurrence during certain years.

__Hint:__ Leave aside the rows with '_PRENOMS_RARES'.

In [ ]:
# your code here


# Getting help

- [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/)

- [plotly](https://plotly.com/python/) for animated plots

- [stackoverflow](https://stackoverflow.com)

- [stackoverflow: pandas](https://stackoverflow.com/questions/tagged/pandas)

- [stackoverflow: plotly+python](https://stackoverflow.com/questions/tagged/plotly+python)

- The US `babynames` analogue of the INSEE file has been a playground for data scientists,
 see [https://github.com/hadley/babynames](https://github.com/hadley/babynames)

- Don't Repeat Yourself (DRY) principle  at [Wikipedia](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)